<a href="https://colab.research.google.com/github/Josh-Been/Recent-Homiletical-Thought/blob/master/RHT_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# Importing Libraries
import pandas as pd, zipfile, numpy as np, os.path
from google.colab import files

# Browse/Upload File
if not os.path.isfile('RHT Theses and Dissertations.zip'):
    up=files.upload()
    with zipfile.ZipFile(next(iter(up)), 'r') as zip_ref:
        zip_ref.extractall()

df_list=[]

for excel in zip_ref.namelist():
    xl = pd.ExcelFile(excel)

    # Pass Sheet Dataframes into df_list   
    for item in xl.sheet_names:
        df=xl.parse(item)
        
        df = df.replace(np.nan, '', regex=True)
        
        df['sheet']=item
        if 'Article' in excel:
            df['tabletype']='Journal Article'
            df['google']='https://scholar.google.com/scholar?hl=en&as_sdt=0%2C44&q='+df['ArticleTitle']
            df['worldcat']='https://www.worldcat.org/search?q=ti:'+df['JournalTitle']
            df['Title']=df['ArticleTitle']
            df['PubDate']=df['JournalYear']
            df['Journal/Pub/Issuer']=df['JournalTitle']
        elif 'Book' in excel:
            df['tabletype']='Book'
            df['google']='https://scholar.google.com/scholar?hl=en&as_sdt=0%2C44&q='+df['Title']
            df['worldcat']='https://www.worldcat.org/search?q=ti:'+df['Title']
            df['Journal/Pub/Issuer']=df['Publisher']
        elif 'Dissertation' in excel:
            df['tabletype']='Thesis/Dissertation'
            df['google']='https://scholar.google.com/scholar?hl=en&as_sdt=0%2C44&q='+df['Title']
            df['worldcat']='https://www.worldcat.org/search?q=ti:'+df['Title']
            df['PubDate']=df['Year']
            df['Annotation']=''
            df['Journal/Pub/Issuer']=df['Institution']+' - '+df['Degree']
        df['author']=df['AuthorFirstName']+' '+df['AuthorMiddleName']+' '+df['AuthorLastName']
        bin_list=[]
        anno_list=[]
        cite_list=[]

        # Some data porocessing could not happen on the dataframe level, so here iterates row by row
        for index, row in df.iterrows():
            if isinstance(row['PubDate'], int) or isinstance(row['PubDate'], float):
                if row['PubDate'] is None:
                    bin_list.append('n/a')
                elif row['PubDate']<1925:
                    bin_list.append('< 1925')
                elif row['PubDate']<1950:
                    bin_list.append('1925-49')
                elif row['PubDate']<1975:
                    bin_list.append('1950-74')
                elif row['PubDate']<2000:
                    bin_list.append('1975-1999')
                elif row['PubDate']>=2000:
                    bin_list.append('2000 >')
                else:
                    bin_list.append('n/a')
            else:
                bin_list.append('n/a')
            df['search_all']=df['tabletype']+' '+df['Title']+' '+df['Journal/Pub/Issuer']+' '+df['author']+' '+df['Annotation']
            if row['tabletype']=='Journal Article':
                citation='https://josh-been.github.io/Recent-Homiletical-Thought/rht_citations.html#/|SBL Citation Style -- '+row['AuthorLastName']+', '+row['AuthorFirstName']+'. \"'+row['ArticleTitle']+'.\" '+row['JournalTitle']+' '+str(row['JournalVolume'])+' ('+str(row['JournalYear'])+'): '+str(row['Pages'])
                citation=citation.replace('   .','.').replace('  .','.').replace(' .','.').replace('.. \"','. \"')
                cite_list.append(citation)
            else:
                cite_list.append('')

        df['DateBin']=bin_list
        df['cites']=cite_list
        df_list.append(df)

# Concatenate Sheets
new_df = pd.concat(df_list, sort=True)
new_df = new_df.reset_index()

# Export to XLSX
new_df.to_excel('concatenated.xlsx',encoding='utf-8-sig')
# new_df.to_json('concatenated.json')

print('\n >> Done!')